# Folder-wise Fast Microscopy Picture Analysis
The following code opens a folder of the Keyence Microscope and automatically sets ROIs to the darkest and brightest portions of the picture. Consequently it removes the background through the ROIs at the darkest positions and calculates the mean fluorescence at the brighter spots. 

In [ ]:
## Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import tkinter as tk
import imagecodecs
from tkinter.filedialog import askopenfilename,askdirectory
import os
from skimage import filters, measure, morphology, draw
import math
import pandas as pd
from PIL import Image

In [ ]:
# Use tkinter to open a file dialog and select the folder with images
def select_folder():
    root = tk.Tk()
    root.withdraw()  # We don't want a full GUI, so keep the root window from appearing
    folder_path = askdirectory(title="Select Folder with Channel TIFF Images")
    root.destroy()
    return folder_path

In [ ]:
# Use tkinter to open a file dialog and select multiple folders
def select_folders():
    root = tk.Tk()
    root.withdraw()  # We don't want a full GUI, so keep the root window from appearing
    folder_list = []
    
    while True:
        folder = askdirectory(title="Select Folder Containing Channel TIFF Images")
        if folder:
            folder_list.append(folder)
        else:
            break  # User cancelled the dialog
    
    root.destroy()
    return list(set(folder_list))  # Remove duplicates


# Function Definitions

In [ ]:
# Normalize images for display
def normalize(image):
    return (image - np.min(image)) / (np.max(image) - np.min(image))

In [ ]:
def background_ROI_substraction(image, background_threshold = None, display_rois = True):
    n_channels = image.shape[2]

    # Initialize empty list to store background values
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    
    # Initialize empty image to store background subtracted image
    background_subtracted_image = np.copy(image)

    background_threshold_temp = background_threshold
    
    for channel_index in range(n_channels):
        channel = image[:, :, channel_index]
        # Define background threshold, by standard deviation from the mean, if not provided
        if background_threshold_temp == None:    
            background_threshold_temp = np.mean(channel) - 0.2 * np.std(channel)
            if background_threshold_temp < 0:
                background_threshold_temp = np.min(channel) + 0.01 * np.std(channel)

        background_thresh = channel < background_threshold_temp
        print(f'Background threshold for channel {channel_index+1}: {background_threshold}')

        # Label the background regions
        background_labels = measure.label(background_thresh)
        background_props = measure.regionprops(background_labels)

        # Create bounding boxes around the detected background contours to define background ROIs
        background_rois = []

        for prop in background_props:
            minr, minc, maxr, maxc = prop.bbox[:4]
            background_rois.append((minc, minr, maxc - minc, maxr - minr))

        # Calculate mean background values for the channel
        for roi in background_rois:
            x, y, w, h = roi
            roi_area = channel[y:y+h, x:x+w]
            mean_background = np.mean(roi_area)
            background_values[f'Channel {channel_index+1}'].append(mean_background)
        
        if display_rois:
            # Display background ROIs on the current channel
            roi_image = np.stack([normalize(channel)]*3, axis=-1)  # Convert to RGB
            for roi in background_rois:
                x, y, w, h = roi
                rr, cc = draw.rectangle_perimeter((y, x), extent=(h, w), shape=channel.shape)
                roi_image[rr, cc] = [1, 0, 0]  # Red for ROIs

            plt.imshow(roi_image)
            plt.title(f'Background ROIs on Channel {channel_index + 1}')
            plt.show()
    
    # Subtract background from the entire image
    mean_background_value = []
    for channel_index in range(n_channels):
        mean_background_value.append(np.mean(background_values[f'Channel {channel_index+1}']))
        print(f'Mean background value for channel {channel_index+1}: {mean_background_value[channel_index]}')
        background_subtracted_image[:, :, channel_index] -= np.minimum(math.floor(mean_background_value[channel_index]), background_subtracted_image[:, :, channel_index])

    return background_values, mean_background_value, background_subtracted_image

In [ ]:
def background_ROI_substraction_single_ch(image, background_threshold = None, channel_of_interest = 0, display_rois = True):
    
    n_channels = image.shape[2]

    # Initialize empty list to store background values
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    
    # Initialize empty image to store background subtracted image
    bg_subs_image_single_ch = image[:,:,channel_of_interest]

    # Define background threshold, by standard deviation from the mean, if not provided
    if background_threshold is None:
            background_threshold = np.mean(bg_subs_image_single_ch) - 0.2 * np.std(bg_subs_image_single_ch)
            if background_threshold < 0:
                background_threshold = np.min(bg_subs_image_single_ch) + 0.01 * np.std(bg_subs_image_single_ch)
            print(f'Background threshold for channel {channel_of_interest+1}: {background_threshold}')
    
    # Find minima below a threshold to define background ROIs

    background_thresh = bg_subs_image_single_ch < background_threshold

    # Label the background regions
    background_labels = measure.label(background_thresh)
    background_props = measure.regionprops(background_labels)

    # Create bounding boxes around the detected background contours to define background ROIs
    background_rois = []

    for prop in background_props:
        minr, minc, maxr, maxc = prop.bbox[:4]
        background_rois.append((minc, minr, maxc - minc, maxr - minr))
    
    if display_rois: 
        # Display background ROIs on channel of interest
        background_roi_image = np.stack([normalize(bg_subs_image_single_ch)]*3, axis=-1)  # Convert to RGB

        for roi in background_rois:
            x, y, w, h = roi
            background_roi_image[y:y+h, x:x+w] = [1, 0, 0]  # Red for background ROIs

        plt.imshow(background_roi_image)
        plt.title(f'Background ROIs on Channel of Interest {channel_of_interest + 1}')
        plt.show()

    # Calculate mean background values for each channel
    for channel_index in range(n_channels):
        channel = image[:, :, channel_index]
        
        for roi in background_rois:
            x, y, w, h = roi
            roi_area = channel[y:y+h, x:x+w]
            mean_background = np.mean(roi_area)
            background_values[f'Channel {channel_index+1}'].append(mean_background)

    mean_background_value = []
    # Subtract background from the entire image
    background_subtracted_image = np.copy(image)
    
    for channel_index in range(n_channels):
        mean_background_value.append(np.mean(background_values[f'Channel {channel_index+1}']))
        background_subtracted_image[:, :, channel_index] -= np.nan_to_num(np.minimum(math.floor(mean_background_value[channel_index]), background_subtracted_image[:, :, channel_index]))
    
    return background_values, mean_background_value, background_subtracted_image

In [ ]:
def load_image(base_name):

    # Load the images
    channel1_path = base_name + "_CH1.tif"
    channel2_path = base_name + "_CH2.tif"
    channel3_path = base_name + "_CH3.tif"
    channel4_path = base_name + "_CH4.tif"

    channel1_raw = tiff.imread(channel1_path)
    channel2_raw = tiff.imread(channel2_path)
    channel3_raw = tiff.imread(channel3_path)
    channel4_raw = tiff.imread(channel4_path)

    channel1 = channel1_raw[:, :, 2] # Blue Channel  
    channel2 = channel2_raw[:, :, 1] # Green Channel
    channel3 = channel3_raw[:, :, 0] # Red Channel
    channel4 = channel4_raw[:, :, 0] # Far Red Channel

    # Combine channels into a single multi-channel image for convenience
    return np.stack((channel1, channel2, channel3, channel4), axis=-1)
    

In [ ]:
def process_images(base_name, lower_thresh_chan, upper_thresh, background_threshold = 0, radius_factor = 10, mask_channel = 1, channel_of_interest = 1, single_ch_background = True):
    '''
    # Function to process a single set of 4-channel images and return mean fluorescence values, mean background values, and number of positive and negative results

            Parameters:
                base_name (string): The path to the folder containing the 4-channel images
                lower_thresh_chan (int): A list of 4 integers, the lower threshold for each channel
                upper_thresh (int): The upper threshold for the channel of interest
                background_threshold (int): The threshold for the background values, if left empty, it will be 2 standard deviations below the mean
                radius_factor (int): The radius of the circular ROIs around detected points
                channel_of_interest (int): The channel of interest, default is 1

            Returns:
                mean_fluorescence (dict): A dictionary containing the mean fluorescence values for each channel
                mean_fluorescence_value (float): The mean fluorescence value for each channel
                background_values (dict): A dictionary containing the mean background values for each channel
                mean_background_value (float): The mean background value for the channel of interest
                positive_results (dict): A dictionary containing the number of positive results for each channel
                negative_results (dict): A dictionary containing the number of negative results for each channel
                corrected_total_fluorescence (dict): A dictionary containing the corrected total cell fluorescence for each channel
    '''
    channel_of_interest -= 1
    mask_channel -= 1

    image = load_image(base_name)

    n_channels = image.shape[2]

    # Initialize dictonaries to store the mean fluorescence and background values 
    mean_fluorescence = {f'Channel {i+1}': [] for i in range(n_channels)}
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    positive_results = {f'Channel {i+1}': [] for i in range(n_channels)}
    corrected_total_fluorescence = {f'Channel {i+1}': [] for i in range(n_channels)}

    # Subtract background from the image and display the background ROIs on the channel of interest
    if single_ch_background:
        background_values, mean_background_value, background_subtracted_image = background_ROI_substraction_single_ch(image, background_threshold,channel_of_interest)
    else:
        background_values, mean_background_value, background_subtracted_image = background_ROI_substraction(image, background_threshold)

    # Display the histogram of the background subtracted image    
    fig, axs = plt.subplots(1, n_channels , figsize=(n_channels*4, 10))

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.hist(image[:, :, channel_index].ravel(), bins=256, color='gray', alpha=0.75)
        ax.set_title(f"Histogram for Channel {channel_index+1}")
        ax.set_xlabel("Pixel intensity")
        ax.set_ylabel("Frequency")
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.axvline(mean_background_value[channel_index], color='r', linestyle='dashed', linewidth=1)
        ax.axvline(lower_thresh_chan[channel_index] * np.std(image[:, :, channel_index]) + np.mean(image[:, :, channel_index]), color='g', linestyle='dashed', linewidth=1)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
    
    # output_path = base_name + "_0_histogram.png"
    # plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.savefig(base_name + "_0_histogram.fig", format='matplotlib')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    
    plt.show()
    
    hist_background_image = np.zeros_like(background_subtracted_image)

    for channel in range(n_channels):
        hist_background_image[:,:,channel] = np.where(background_subtracted_image[:,:,channel] < 1, 1, background_subtracted_image[:,:,channel])


    # Apply thresholds to find maximum values in the background-subtracted depending on channel of interest, avoiding very bright spots
    # Mask out very bright spots
    channel_thresh = 3 * np.std(background_subtracted_image[:, :, mask_channel]) + np.mean(background_subtracted_image[:, :, mask_channel])
    print(f'Channel threshold for mask channel {mask_channel+1}: {channel_thresh}')
    thresh = (background_subtracted_image[:, :, mask_channel] > channel_thresh) & (background_subtracted_image[:, :, mask_channel] < upper_thresh)

    # Label the thresholded regions and return the number of cells
    labels = measure.label(thresh)
    props = measure.regionprops(labels)

    # Create circular ROIs around detected points
    rois = []

    for prop in props:
        y, x = prop.centroid
        radius = radius_factor 
        rois.append((int(x), int(y), int(radius)))
    
    # Display circular ROIs on the background-subtracted masked channel
    roi_image = np.stack([normalize(background_subtracted_image[:, :, mask_channel])]*3, axis=-1)  # Convert to RGB

    for roi in rois:
        x, y, radius = roi
        rr, cc = draw.disk((y, x), radius, shape=roi_image.shape)
        roi_image[rr, cc] = [0, 1, 0]  # Green for ROIs

    plt.imshow(roi_image)
    plt.title(f'ROIs on Background-Subtracted Mask_Channel {mask_channel+1}')
    plt.show()

    # Calculate mean fluorescence values for each channel and check if the signal is present
    for channel_index in range(n_channels):
        channel = background_subtracted_image[:, :, channel_index]
        channel_thresh = lower_thresh_chan[channel_index] * np.std(channel) + np.mean(channel)

        for roi in rois:
            x, y, radius = roi
            rr, cc = draw.disk((y, x), radius, shape=channel.shape)
            roi_area = channel[rr, cc]
            mean_value = np.mean(roi_area)
            integrated_density = np.sum(roi_area)
            area_of_cell = len(rr)
            corrected_fluorescence = integrated_density - (area_of_cell * mean_background_value[channel_index])

            if mean_value > channel_thresh:
                positive_results[f'Channel {channel_index+1}'].append((x, y, radius))
                mean_fluorescence[f'Channel {channel_index+1}'].append(mean_value)
                corrected_total_fluorescence[f'Channel {channel_index+1}'].append(corrected_fluorescence)
            else:
                positive_results[f'Channel {channel_index+1}'].append(None)
                corrected_total_fluorescence[f'Channel {channel_index+1}'].append(None)
    
    fig, axs = plt.subplots(1, n_channels , figsize=(n_channels *5,20))

    axs[0].imshow(normalize(image[0]), cmap='gray')
    axs[1].imshow(normalize(image[1]), cmap='gray')
    axs[2].imshow(normalize(image[2]), cmap='gray')
    axs[3].imshow(normalize(image[3]), cmap='gray')

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.set_title(f'Channel {channel_index+1} (Raw)')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
        ax.axis('off')
    
    output_path = base_name + "_1_colored.png"
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.show()

    # Display positive ROIs for each channel
    fig, ax = plt.subplots(1, 4, figsize=(n_channels * 5, 20))

    for channel_index in range(n_channels):
        channel_image = np.stack([normalize(background_subtracted_image[:, :, channel_index])]*3, axis=-1)  # Convert to RGB

        for roi in positive_results[f'Channel {channel_index+1}']:
            if roi is not None:
                x, y, radius = roi
                rr, cc = draw.disk((y, x), radius, shape=channel_image.shape)
                channel_image[rr, cc] = [0, 1, 0]  # Green for positive ROIs

        ax[channel_index].imshow(channel_image)
        ax[channel_index].set_title(f'Positive ROIs on Channel {channel_index + 1}')
        ax[channel_index].axis('off')

    output_path = base_name + "_2_ROIs.png"
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)    
    plt.show()
    

    return mean_fluorescence, background_values, mean_background_value, positive_results, corrected_total_fluorescence

In [ ]:
def stitch_images(folder_path):
    # List of channels
    channels = ['CH1', 'CH2', 'CH3', 'CH4']
    
    # Find all image files in the folder
    all_files = os.listdir(folder_path)
    
    for channel in channels:
        # Filter out files for the current channel
        channel_files = sorted([f for f in all_files if f.endswith(f'_{channel}.tif')])
        
        if len(channel_files) != 4:
            print(f"Error: Expected 4 images for {channel}, but found {len(channel_files)}.")
            continue
        
        # Load images for the current channel
        images = []
        for file_name in channel_files:
            image_path = os.path.join(folder_path, file_name)
            img = Image.open(image_path)
            images.append(np.array(img))

        # Check if all images have the same shape
        assert all(img.shape == images[0].shape for img in images), "All images must have the same shape"
        
        # Arrange images in a 2x2 grid
        upper_row = np.hstack((images[0], images[1]))
        lower_row = np.hstack((images[2], images[3]))
        final_image_array = np.vstack((upper_row, lower_row))

        # Convert back to an image
        final_image = Image.fromarray(final_image_array)

        # Save the final stitched image
        final_image.save(os.path.join(folder_path, f'stitched_{channel}.tif'))
        print(f"Stitched image for {channel} saved as stitched_{channel}.tif")

# Main Workflow Point

In [ ]:
# Alternative method to select the folder (open main folder with all subfolders)
main_folder_path = select_folder()

In [ ]:
# Try out for stiching
# stitch_images(folder_path) # Channel 3 crashing the code, revise

In [ ]:
# Save memory by closing all currently open figures
plt.close('all')

# Open the folder which contains the images
#folder_path = select_folder()

# Process all sets of images in the folder and collect the results
results = []

# Thresholds for ROI detection and background exclusion
lower_thresh_factor = [2, 3, 2, 2]  # Adjust this value based on your needs
upper_thresh = 60000  # Adjust this value to exclude very bright spots
background_threshold = 0  # Adjust this value based on your needs
radius_factor = 10  # Factor to determine the radius of the circular ROIs
mask_channel = 1  # Channel to use for masking bright spots
channel_of_interest = 1  # Channel of interest for Background ROI detection

for root, dirs, files in os.walk(main_folder_path):
    for filename in files:
        if filename.endswith("_CH1.tif"):
            base_name = os.path.join(root, filename.replace("_CH1.tif", ""))
            try:
                mean_fluorescence, background_values, mean_background_value, positive_results, corrected_total_fluorescence = process_images(base_name, 
                                                                                                            lower_thresh_factor, 
                                                                                                            upper_thresh, 
                                                                                                            background_threshold, 
                                                                                                            radius_factor, 
                                                                                                            mask_channel, 
                                                                                                            channel_of_interest, 
                                                                                                            single_ch_background = True)
            except:
                print(f"Error processing {base_name}. Skipping...")
                pass    
            # Collect results into a list of dictionaries for easy conversion to DataFrame
            result = {'Base Name': base_name}
            for channel, values in mean_fluorescence.items():
                result[f'{channel} Fluorescence mean value'] = np.mean(values)
                # result[f'{channel} Background Values'] = background_values[channel]
                result[f'{channel} Mean Background'] = np.mean(background_values[channel])
                result[f'{channel} Positive Results'] = sum(x is not None for x in positive_results[channel]) 
                result[f'{channel} Negative Results'] = sum(x is None for x in positive_results[channel])
                result[f'{channel} Corrected Total Fluorescence'] = np.mean([x for x in corrected_total_fluorescence[channel] if x is not None])
            results.append(result)

    # Convert the results to a DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(os.path.join(root, 'mean_fluorescence_results.csv'), index=False)

    print("Processing complete. Results saved to mean_fluorescence_results.csv")
